# Práctica Guiada de Laboratorio 10 - Curso IA 2025-1 Grupo 1 EPISW
# Ejercicio propuesto - c-means
### Prof. Rolando A. Maguiña Pérez

# Introducción
En la presente práctica, a realizarse el 19 de Junio del 2025, se abordará el problema denominado `Segmentación de clientes` mediante la técnica  del  `Machine Learning` denominada `c-means`. Luego de la presentación del problema, se plantearán algunas preguntas que los alumnos deberán resolver.

En esas actividades se usará la biblioteca de aprendizaje automático para el lenguaje de programación Python llamada  [scikit-learn](https://scikit-learn.org/stable/).

**Nota: esta Práctica podrá desarrollarse en grupos de tres alumnos.**

# Segmentación de clientes
## Planteamiento del problema
El objetivo de este problema es segmentar a los clientes de un distribuidor mayorista en función de su gasto anual en diversas categorías de productos, como leche, comestibles, región, etc.

Los datos están alojados en el repositorio de aprendizaje automático de la Universidad de Irving y consta de 440 muestras. Cada registro de la base de datos tiene ocho atributos numéricos.

1. Channel
2. Region
3. Fresh
4. Milk
5. Grocery
6. Frozen
7. Detergents_Paper
8. Delicassen

El archivo con el dataset es `Wholesale customers data.csv`

### Carga de los módulos/bibliotecas y la data
**Importe las librerias que generalmente usa para el análisis de datos.**

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
import seaborn as sns

### Obtener los datos


**Lea el archivo `Wholesale customers data.csv`.**

In [6]:
data= pd.read_csv('Wholesale customers data.csv')

**Revise los primeros registros de los datos**

In [7]:
data.head()

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,2,3,12669,9656,7561,214,2674,1338
1,2,3,7057,9810,9568,1762,3293,1776
2,2,3,6353,8808,7684,2405,3516,7844
3,1,3,13265,1196,4221,6404,507,1788
4,2,3,22615,5410,7198,3915,1777,5185


Tenemos los detalles de gasto de los clientes en diferentes productos, como leche, comestibles, congelados, detergentes, etc. Ahora, debemos segmentar a los clientes según la información proporcionada. Antes de hacerlo, extraigamos algunas estadísticas relacionadas con los datos:

**Aplique los métodos info() y describe() en los datos.**

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   Channel           440 non-null    int64
 1   Region            440 non-null    int64
 2   Fresh             440 non-null    int64
 3   Milk              440 non-null    int64
 4   Grocery           440 non-null    int64
 5   Frozen            440 non-null    int64
 6   Detergents_Paper  440 non-null    int64
 7   Delicassen        440 non-null    int64
dtypes: int64(8)
memory usage: 27.6 KB


In [10]:
data.describe()

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
count,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000
mean,1.322727,2.543182,12000.297727,5796.265909,7951.277273,3071.931818,2881.493182,1524.870455
std,0.468052,0.774272,12647.328865,7380.377175,9503.162829,4854.673333,4767.854448,2820.105937
min,1.000000,1.000000,3.000000,55.000000,3.000000,25.000000,3.000000,3.000000
25%,1.000000,2.000000,3127.750000,1533.000000,2153.000000,742.250000,256.750000,408.250000
50%,1.000000,3.000000,8504.000000,3627.000000,4755.500000,1526.000000,816.500000,965.500000
75%,2.000000,3.000000,16933.750000,7190.250000,10655.750000,3554.250000,3922.000000,1820.250000
max,2.000000,3.000000,112151.000000,73498.000000,92780.000000,60869.000000,40827.000000,47943.000000


Aquí, observamos que hay mucha variación en la magnitud de los datos. Variables como Channel y Region tienen una magnitud baja, mientras que variables como Fresh, Milk, Groceries, etc. tienen una magnitud mayor.

Dado que c-means es un algoritmo basado en la distancia, esta diferencia de magnitud puede crear un problema. Por lo tanto, primero igualemos la magnitud de todas las variables. Escriba el código que permita hacer esto.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)
scaled_data = pd.DataFrame(scaled_data, columns=data.columns)
scaled_data.describe()

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
count,4.400000e+02,4.400000e+02,4.400000e+02,440.000000,4.400000e+02,4.400000e+02,4.400000e+02,4.400000e+02
mean,1.614870e-17,3.552714e-16,-3.431598e-17,0.000000,-4.037175e-17,3.633457e-17,2.422305e-17,-8.074349e-18
std,1.001138e+00,1.001138e+00,1.001138e+00,1.001138,1.001138e+00,1.001138e+00,1.001138e+00,1.001138e+00
min,-6.902971e-01,-1.995342e+00,-9.496831e-01,-0.778795,-8.373344e-01,-6.283430e-01,-6.044165e-01,-5.402644e-01
25%,-6.902971e-01,-7.023369e-01,-7.023339e-01,-0.578306,-6.108364e-01,-4.804306e-01,-5.511349e-01,-3.964005e-01
50%,-6.902971e-01,5.906683e-01,-2.767602e-01,-0.294258,-3.366684e-01,-3.188045e-01,-4.336004e-01,-1.985766e-01
75%,1.448652e+00,5.906683e-01,3.905226e-01,0.189092,2.849105e-01,9.946441e-02,2.184822e-01,1.048598e-01
max,1.448652e+00,5.906683e-01,7.927738e+00,9.183650,8.936528e+00,1.191900e+01,7.967672e+00,1.647845e+01


## Creación del clúster K Means

¡Ahora es el momento de crear las etiquetas del Cluster!

**Importar KMeans de SciKit Learn.**

In [ ]:
#2

**Crea una instancia de un modelo de c-means con 2 clústeres.**
Use el algoritmo de inicialización denominado `k-means++`

In [ ]:
#2

Se han inicializado dos clústeres y notar que la inicialización no ha sido aleatoria. Se ha utilizado la inicialización `k-means++`, que generalmente produce mejores resultados.

**Ajuste el modelo a todos los datos sin excepción alguna.**

In [ ]:
#2
#Usa scaled_data en lugar de data

**¿Cuáles son los centros del grupo de los vectores?**

In [ ]:
#2

**Estudiar e implementar un procedimiento para determinar el número óptimo de clústeres. Sugerencia: use el denominado método del codo.**

In [ ]:
#3

**Aplique ahora el algoritmo usado para el problema de la predicción de préstamo (notebook PGL10_IA_2025-1_EPISW_c-means_OK.ipynb) al caso de segmentación de clientes. Compare los resultados que obtiene con los que obtuvo con sklearn**

In [ ]:
#4

## Instrucciones para el envío de la solución

La solución de la "Práctica Guiada de Laboratorio 10 - IA 2025-1 G1 EPISW" podrá enviarse al correo electrónico rmaguinacursos@gmail.com, hasta las 23:59 h del Domingo 22 de Junio del 2025, en un archivo con extensión .ipynb.

El documento deberá tener las sgtes características:

- Nombre del archivo: solPGL10_IA_2025-1_G1_EPISW_nombre-apellidos_alumno1_..._nombre-apellidos_alumno3.ipynb.

- Todas las preguntas de la Práctica deben responderse en un cuaderno computacional interactivo (**Sugerencia**: obtener una copia de este documento y desarrollar en éllas las respectivas soluciones); la solución a cada pregunta debe registrarse en una celda debajo del planteamiento de la misma, mencionando explícitamente como subtítulo: "Solución del ejercicio n", donde "n" corresponde al número del ejercicio.